In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import tensorflow as tf
import keras.backend as K

from keras.layers import Input, Dense, Dropout, Activation, Add, Concatenate, Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, MaxPool2D, Flatten, BatchNormalization
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from keras.datasets import cifar10

from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score,  roc_curve, auc, confusion_matrix 
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, RobustScaler, StandardScaler

In [2]:
(X_train_images, y_train_labels), (X_test_images, y_test_labels) = cifar10.load_data()

print('X_train_images.shape: ', X_train_images.shape)
print('X_test_images.shape: ', X_test_images.shape)
print('y_train_labels.shape: ', y_train_labels.shape)
print('y_test_labels.shape: ', y_test_labels.shape)

170508288/170498071 [==============================] - 12s 0us/step
X_train_images.shape:  (50000, 32, 32, 3)
X_test_images.shape:  (10000, 32, 32, 3)
y_train_labels.shape:  (50000, 1)
y_test_labels.shape:  (10000, 1)


In [3]:
no_of_classes = 10
dict = {0:'Airplane', 1:'Automobile', 2:'Bird', 3:'Cat', 4:'Deer', 5:'Dog', 6:'Frog', 7:'Horse', 8:'Ship', 9:'Truck'}
img_width, img_height = 32, 32
epochs = 100
batch_size = 512

if K.image_data_format()=='channels_first':
    input_shape=(3, img_width, img_height)
else:
    input_shape=(img_width,img_height,3)

In [4]:
def preprocess_data(X_train_images, X_test_images, y_train_labels, y_test_labels, no_of_classes, val = False):
  #normalize
  X_train = X_train_images.astype('float32')/255.0
  X_test = X_test_images.astype('float32')/255.0

  if val == True:
    
    #split training data further into training and validation set
    (X_train, X_valid) = X_train[:20000], X_train[-8500:]
    (y_train_labels, y_valid_labels) = y_train_labels[:20000], y_train_labels[-8500:]

    print('X_train.shape: ', X_train.shape)
    print('X_valid.shape: ', X_valid.shape)
    print('y_train_labels.shape: ', y_train_labels.shape)
    print('y_valid_labels.shape: ', y_valid_labels.shape)
    
    # The target variable is converted to one-hot encoded data 
    y_train = to_categorical(y_train_labels, no_of_classes)
    y_valid = to_categorical(y_valid_labels, no_of_classes)
    y_test = to_categorical(y_test_labels, no_of_classes)
    print('y_train.shape: ', y_train.shape)
    print('y_test.shape: ', y_test.shape)
    return ((X_train, y_train), (X_test, y_test), (X_valid, y_valid), (y_train_labels, y_test_labels))

  else:
    
    print('X_train.shape: ', X_train.shape)
    print('y_train_labels.shape: ', y_train_labels.shape)
    
    # The target variable is converted to one-hot encoded data 
    y_train = to_categorical(y_train_labels, no_of_classes)
    y_test = to_categorical(y_test_labels, no_of_classes)
    print('y_train.shape: ', y_train.shape)
    print('y_test.shape: ', y_test.shape)
    
    return ((X_train, y_train), (X_test, y_test), (y_train_labels, y_test_labels))

In [5]:
(X_train, y_train), (X_test, y_test) , (y_train_labels,y_test_labels) = preprocess_data(X_train_images, X_test_images, y_train_labels, y_test_labels, no_of_classes, val = False)

X_train.shape:  (50000, 32, 32, 3)
y_train_labels.shape:  (50000, 1)
y_train.shape:  (50000, 10)
y_test.shape:  (10000, 10)


In [ ]:
def create_block(input, chs): ## Convolution block of 1 layer
    x = input
    for i in range(1):
        x = Conv2D(chs, 3, padding="same", activation='relu')(x)
        x = BatchNormalization()(x)
    return x

In [ ]:
def showOrigDec(orig, dec, num=10):  ## function used for visualizing original and reconstructed images of the autoencoder model
    n = num
    plt.figure(figsize=(20, 4))

    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i+1)
        plt.imshow(orig[300*i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i +1 + n)
        plt.imshow(dec[300*i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
def show_test(m, d):  ## function used for visualizing the predicted and true labels of test data
    plt.figure(figsize =(40,8))
    for i in range(5):
        ax = plt.subplot(1, 5, i+1)
        test_image = np.expand_dims(d[1810*i+5], axis=0)
        test_result = m.predict(test_image)
        plt.imshow(x_test_final[1810*i+5])
        index = np.argsort(test_result[0,:])
        plt.title("Pred:{}, True:{}".format(dict[index[9]], dict[y_test_final[1810*i+5][0]]))
    plt.show()

In [ ]:
def report(predictions): ## function used for creating a classification report and confusion matrix
    cm=confusion_matrix(y_test_one_hot.argmax(axis=1), predictions.argmax(axis=1))
    print("Classification Report:\n")
    cr=classification_report(y_test_one_hot.argmax(axis=1),
                                predictions.argmax(axis=1), 
                                target_names=list(dict.values()))
    print(cr)
    plt.figure(figsize=(12,12))
    sns.heatmap(cm, annot=True, xticklabels = list(dict.values()), yticklabels = list(dict.values()), fmt="d")

In [ ]:
def conv_ae():
    input = Input((32,32,3))
    
    # Encoder
    block1 = create_block(input, 8)
    x = MaxPool2D(2)(block1)
    block2 = create_block(x, 16)
    x = MaxPool2D(2)(block2)
    
    #Middle
    middle = create_block(x, 32)
    
    # Decoder
    up1 = UpSampling2D((2,2))(middle)
    block3 = create_block(up1, 16)
    #up1 = UpSampling2D((2,2))(block3)
    up2 = UpSampling2D((2,2))(block3)
    block4 = create_block(up2, 8)
    #up2 = UpSampling2D((2,2))(block4)
    
    # output
    x = Conv2D(3, 1)(up2)
    output = Activation("sigmoid")(x)

    return Model(input, middle), Model(input, output)

In [ ]:
def run_ae(m): ## function for running autoencoder
  encoder, model = conv_ae()
    
  return encoder, model

In [ ]:
encoder_ae, model_ae = run_ae('ae')
model_ae.compile(optimizer = 'Adam', loss='mse')
model_ae.summary()

In [ ]:
start = timeit.default_timer()
history = model_ae.fit(X_train, X_train, 
                       batch_size=80,
                       epochs=40,
                       verbose=1,
                       shuffle=True)
stop = timeit.default_timer()
print(history)
print('Time to train Autoencoder:', stop-start)

In [ ]:
model_ae.save('model_AE.h5')

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='lower right')
plt.show()

In [ ]:
recon_test_ae = model_ae.predict(X_test)
#recon_valid_ae = model_ae.predict(X_valid)

In [ ]:
showOrigDec(X_test, recon_test_ae)

SVM

In [ ]:
# Data 
gist_train_ae = encoder_ae.predict(X_train)
#gist_valid_ae = encoder_ae.predict(X_valid)
gist_test_ae = encoder_ae.predict(X_test)

In [ ]:
svm_clf = SVC(C=10, gamma=0.1, kernel='poly')

In [ ]:
# Training SVM - Parameters are initialised 
start = timeit.default_timer()
svm_clf.fit(gist_train_ae.reshape(gist_train_ae.shape[0], -1), y_train_labels.reshape(-1))
stop = timeit.default_timer()
print("Total time to train SVM:", stop-start)

In [ ]:
# Prediction on SVM
start = timeit.default_timer()
y_pred = svm_clf.predict(gist_test_ae.reshape(gist_test_ae.shape[0], -1))
stop = timeit.default_timer()
print("Total inference time for SVM:", stop-start)

In [ ]:
svm_report = metrics.classification_report(y_test_labels, y_pred, digits=5, target_names=dict.values())
print("Classification report: \n", svm_report)

# display confusion matrix
cm_svm = confusion_matrix(y_test_labels, y_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm_svm, annot=True, xticklabels = list(dict.values()), yticklabels = list(dict.values()), fmt="d")